In [2]:
!pip install faker -q

^C
ERROR: Operation cancelled by user


In [3]:
!pip install -U pip -q

^C
ERROR: Operation cancelled by user


In [4]:
!pip install -U accelerate transformers bitsandbytes 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 80.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [5]:
!pip install --upgrade transformers 

In [4]:
import numpy as np
import pandas as pd 
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import seaborn as sns
import random
from faker import Faker
import json
from tqdm.notebook import tqdm
# from ctransformers import AutoModelForCausalLM

2024-04-15 03:15:34.029308: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 03:15:34.029408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 03:15:34.164902: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
class Config:
    Mixtral_8x7b = 'mistralai/Mixtral-8x7B-v0.1'
    misteral_instruct = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'
    alpindale_mistral = 'alpindale/Mistral-7B-v0.2-hf'

In [6]:
train_df = pd.read_json( '/kaggle/input/pii-detection-removal-from-educational-data/train.json')
test_df = pd.read_json( '/kaggle/input/pii-detection-removal-from-educational-data/test.json')

In [7]:
train_df.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [8]:
train_df.shape

(6807, 5)

In [9]:
train_df.labels.explode().value_counts()

labels
O                   4989794
B-NAME_STUDENT         1365
I-NAME_STUDENT         1096
B-URL_PERSONAL          110
B-ID_NUM                 78
B-EMAIL                  39
I-STREET_ADDRESS         20
I-PHONE_NUM              15
B-USERNAME                6
B-PHONE_NUM               6
B-STREET_ADDRESS          2
I-URL_PERSONAL            1
I-ID_NUM                  1
Name: count, dtype: int64

In [10]:
faker = Faker()

In [11]:
def generate_fake_data( ):

    countries = ['en_IN', 'en_GB', 'en_CA', 'en_AU' , 'en_IE', 'en_US']
    faker = Faker(random.choice( countries ))
    social_media = [
        'twitter', 'facebook', 'instagram',          
    ]
    sm = random.choice(social_media)
    first_name = faker.first_name()
    last_name = faker.last_name()
    username = faker.profile()['username']
    full_name = f'{first_name}_{last_name}'.lower()
    phone_number = faker.phone_number()
    email = faker.email()
    address = faker.address(),
    personal_id = str(faker.unique.random_int(min=111111, max=999999))
    url = f'https://{sm}/en/{full_name}/'
    return first_name, last_name, username,personal_id, phone_number, address, email, url

In [12]:
for i in tqdm(range( 10)):
    first_name, last_name, username, personal_id, phone_number, address, email, url = generate_fake_data()
#     print( first_name, last_name, personal_id, phone_number, address[0], email, url )
    prompt=f"""
    Generate a short essay using the following details:
    id : {personal_id}
    first name: {first_name}
    last name: {last_name}
    user name: {username}
    phone_number: {phone_number}
    address: {address[0]}
    email: {email}
    url: {url}
    
    """
#     print(prompt)

  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:

prompt=f"""
Generate a short essay using the following details about a person:
{personal_id} {first_name} {last_name} {username} {phone_number} {address[0]} {email} {url}
"""
print( prompt)



Generate a short essay using the following details about a person:
684296 Timothy Davidson sandra40 +61.2.3973.2722 86 Dennis Grange
Lindseyborough, VIC, 2977 mathewsthomas@example.net https://twitter/en/timothy_davidson/



In [14]:
import warnings
warnings.filterwarnings('ignore')

### Generate text via Mistral

In [15]:
import gc
torch.cuda.empty_cache()
gc.collect()

1069

In [16]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
import torch

In [17]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [18]:
model_4bit = AutoModelForCausalLM.from_pretrained( Config.misteral_instruct, device_map="auto",quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(Config.misteral_instruct)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
prompts=[]
for i in tqdm(range( 10)):
    first_name, last_name,username, personal_id, phone_number, address, email, url = generate_fake_data()
    profile = first_name, last_name,username, personal_id, phone_number, address[0], email, url 
    prompt=f"""
    <|im_start|>system
    Generate a story using the following details about a person. Just generate the response nothing else:<|im_end|>
    <|im_start|>user
    {personal_id} {first_name} {last_name} {username} {phone_number} {address[0]} {email} url: {url} 
    Must use this url as a part of the passage. Don't start with 'Once upon a time'.
    <|im_end|>
    <|im_start|>assistant
    """
    prompts.append( {'profile': ';'.join(profile),'prompt': prompt})
    print(prompts)

  0%|          | 0/10 [00:00<?, ?it/s]

[{'profile': 'Terri;Barry;christian94;898160;407-604-9590;USNV Fitzgerald\nFPO AE 35092;michael04@example.com;https://twitter/en/terri_barry/', 'prompt': "\n    <|im_start|>system\n    Generate a story using the following details about a person. Just generate the response nothing else:<|im_end|>\n    <|im_start|>user\n    898160 Terri Barry christian94 407-604-9590 USNV Fitzgerald\nFPO AE 35092 michael04@example.com url: https://twitter/en/terri_barry/ \n    Must use this url as a part of the passage. Don't start with 'Once upon a time'.\n    <|im_end|>\n    <|im_start|>assistant\n    "}]
[{'profile': 'Terri;Barry;christian94;898160;407-604-9590;USNV Fitzgerald\nFPO AE 35092;michael04@example.com;https://twitter/en/terri_barry/', 'prompt': "\n    <|im_start|>system\n    Generate a story using the following details about a person. Just generate the response nothing else:<|im_end|>\n    <|im_start|>user\n    898160 Terri Barry christian94 407-604-9590 USNV Fitzgerald\nFPO AE 35092 michae

In [32]:
!rm -rf pii_kaggle_compettition_dataset.jsonl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
for idx,chat in tqdm(enumerate(prompts)):
#     print(chat)
    input_ids = tokenizer(chat['prompt'], return_tensors="pt").input_ids.to("cuda")
    generated_ids = model_4bit.generate(input_ids, max_new_tokens=500, temperature=0.1, repetition_penalty=1.1, do_sample=True, eos_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(generated_ids[0][input_ids.shape[-1]:], skip_special_tokens=True, clean_up_tokenization_space=True)
    print(f"Response: {response}")
    with open( 'pii_kaggle_compettition_dataset.jsonl', 'a') as f:
        data = {'id': str(faker.uuid4()),'profile': chat['profile'], 'response': response ,  }  
        json_str = json.dumps( data )
        f.write( json_str+'\n')
    print("")

0it [00:00, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response: 
    Terri Barry was a 40-year-old woman who lived in Las Vegas, Nevada. She was a Christian and had a passion for helping others. Her phone number was 407-604-9590 and she could be reached at her email address [michael04@example.com](mailto:michael04@example.com).

    Terri was an active member of her community and often volunteered at local charities. She was also an avid Twitter user and could be found sharing her thoughts and ideas on her page at [https://twitter/en/terri\_barry/](https://twitter/en/terri_barry/).

    One day, while browsing through her Twitter feed, Terri came across a post from someone who was struggling to make ends meet. The person was a single mother who was trying to provide for her children but was falling behind on bills. Terri immediately felt compelled to help and reached out to the person via direct message.

    Over the next few weeks, Terri worked tirelessly to help the single mother get back on track. She connected her with local resource

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response: Ryan Adams was born on January 4th, 1969 in Rosswilkinson, England. He grew up in a small town and always had a passion for music. As a teenager, he began playing guitar and writing songs. After high school, he moved to London to pursue his dream of becoming a musician.
     Ryan quickly gained recognition for his talent and was signed by a record label. He released several successful albums and toured extensively throughout Europe and North America. Despite his success, Ryan remained humble and down-to-earth.
     In addition to his music career, Ryan is also known for his philanthropic work. He has donated millions of dollars to various charities over the years and is actively involved in many causes close to his heart.
     Ryan's website can be found at <https://facebook/en/ryan_adams/>.



...

Response: Once upon a time, there was a woman named Neysa Upadhyay who lived in Burhanpur, India. She was a kind and caring person who always went out of her way to help others. Neysa had a passion for photography and spent most of her free time capturing beautiful moments through her camera lens.

    Neysa was also an avid traveler and loved exploring new places. She had been to many different countries and had even started her own travel blog where she shared her experiences with others. Her blog had become very popular and she had a large following of people who were inspired by her adventures.

    One day, while browsing through her Facebook page, Neysa came across a post from a friend who was looking for someone to join them on a trip to Europe. Without hesitation, Neysa jumped at the opportunity and booked her tickets.

    When she arrived in Europe, Neysa was amazed by the beauty and culture of the continent. She visited many famous landmarks such as the Eiffel Tower, Big Ben

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response: 
    Cheryl Edwards was a 26 year old woman from Berry Summit, Lake Jamieborough, YT R3C 2C3. She had a passion for photography and spent most of her free time capturing the beauty of nature through her camera lens. Her work had been featured in several local magazines and she had even won a few awards for her stunning shots.

    Cheryl was also an avid traveler and loved exploring new places. She had visited many countries around the world and had made countless memories along the way. Her adventurous spirit and love for photography made her the perfect candidate for a job as a travel photographer.

    One day, while browsing through her favorite website, she stumbled upon a job posting for a travel photographer. The job required traveling to different parts of the world and capturing stunning images of landscapes, people, and cultures. Cheryl knew that this was the opportunity she had been waiting for.

    She immediately applied for the job and included her portfolio of

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response: 
Corey Mann was a young man from South Alex, NT, Australia. He had just turned 25 and was excited to start his new life as an adult. Corey had always been interested in technology and had spent most of his childhood tinkering with gadgets and computers.

As he grew older, Corey's passion for technology only intensified. He studied computer science at university and graduated with honors. After graduation, Corey landed a job at Sony as a software engineer.

In his free time, Corey loved to explore the latest technologies and experiment with new ideas. He was also an avid photographer and often shared his work on Instagram using the handle @corey\_mann.

Despite his busy schedule, Corey always made time for his family and friends. He was especially close to his mother, Joanne Parade, who had always been there for him throughout his life.

Overall, Corey Mann was a bright and talented individual with a passion for technology and a love for life.

Response: 403374 Heather Donovan

In [ ]:
with open( '/kaggle/working/pii_kaggle_compettition_dataset.jsonl', 'r') as f:
    print( f.read())

### End Of Notebook